In [26]:
import pandas as pd
import re
from rapidfuzz import fuzz, process

coaches = pd.read_csv('../data/raw/coaches.csv')
players = pd.read_csv('../data/raw/players.csv')
schools = pd.read_csv('../data/raw/schools.csv')


In [67]:
print(coaches.head())
print(players.head())
print(schools.head())

                  School              Name                    Title
0  University Of Florida  Kevin O'Sullivan               Head Coach
1  University Of Florida   Chuck Jeroloman     Associate Head Coach
2  University Of Florida      Taylor Black          Assistant Coach
3  University Of Florida        David Kopp           Pitching Coach
4  University Of Florida     Michael Byrne  Student Assistant Coach
                  School           Name  Jersey Position Class Year   Height  \
0  University Of Florida  Justin Nadeau       1   INF/OF        Jr.   6' 0''   
1  University Of Florida       Ty Evans       2       OF        Sr.   6' 2''   
2  University Of Florida     Kyle Jones       3       OF        So.   6' 3''   
3  University Of Florida   Cade Kurland       4      INF        Jr.  5' 11''   
4  University Of Florida      Blake Cyr       5   OF/INF        Jr.  5' 11''   

    Weight                   High School  
0  195 lbs  Bartram Trail (Jacksonville)  
1  215 lbs            Lak

In [68]:
counts = players['Name'].value_counts()
multiple_player_names = counts[counts > 1]
multiple_player_names

Name
Joey McMannis      2
Cameron Johnson    2
Name: count, dtype: int64

In [69]:
high_schools = players['High School'].dropna().unique()
len(high_schools)

444

In [70]:
temp_schools = high_schools[:5]
for school in temp_schools:
    if school == 'N/A':
        continue

    matches = process.extract(
        school,
        temp_schools,
        scorer = fuzz.ratio,
        score_cutoff = 80
    )

    print(matches[0])

('Bartram Trail (Jacksonville)', 100.0, 0)
('Lakeland Christian', 100.0, 1)
('North Oconee (Stetson)', 100.0, 2)
('Berkeley Prep', 100.0, 3)
('Windermere (Miami)', 100.0, 4)


In [79]:
def dedup_high_schools(high_schools, canonical_hs, score_cutoff):
    hs_mappings = {}
    for school in high_schools:
        if school == 'N/A':
            continue

        matches = process.extract(
            school,
            canonical_hs,
            scorer = fuzz.partial_ratio,
            score_cutoff = score_cutoff
        )

        if matches:
            hs_mappings[school] = matches[0][0]
        else:
            hs_mappings[school] = school

    return hs_mappings

In [ ]:
for score_cutoff in [95]:
    hs_mappings = {}
    print(hs_mappings)
    canonical_hs = schools[schools['school type'] == 'high school']['name'].unique()
    high_schools = players['High School'].dropna().unique()
    hs_mappings = dedup_high_schools(high_schools, canonical_hs, score_cutoff=score_cutoff)

    for key, vals in hs_mappings.items():
        if key != vals:
            print(f"{key!r} -> {vals!r}")

    print("\n")

{}
'Lakeland Christian (Virginia Tech)' -> 'Lakeland Christian'
'TNXL Academy (Charleston Southern)' -> 'TNXL Academy'
'Liberty HS' -> 'Liberty'
'John Carroll Catholic HS' -> 'Catholic HS'
'Brandon HS' -> 'Brandon'
'Franklin HS' -> 'Franklin'
'Pearland HS' -> 'Pearl'
'Columbia River' -> 'Columbia'
'Buchanan HS' -> 'Buchanan'
'Sherwood HS' -> 'Sherwood'
'Aquinas HS' -> 'Aquinas'
'Liberty-Wentzville' -> 'Liberty'
'East Ridge High School' -> 'East Ridge'
'Austin Prep' -> 'Austin'
'Wayzata HS' -> 'Wayzata'
'Montgomery Lake Creek' -> 'Lake Creek'
'Trussville' -> 'Hewitt-Trussville'
'TNXL Academy [Fla.]' -> 'TNXL Academy'




: 

In [75]:
hs_mappings = {}

for school in high_schools:
    if school == 'N/A':
        continue

    matches = process.extract(
        school,
        canonical_hs,
        scorer = fuzz.partial_ratio,
        score_cutoff = 70
    )

    if matches:
        hs_mappings[school] = matches[0][0]
    else:
        hs_mappings[school] = school

for key, vals in hs_mappings.items():
    if key != vals:
        print(f"{key!r} -> {vals!r}")

'Lakeland Christian (Virginia Tech)' -> 'Lakeland Christian'
'TNXL Academy (Charleston Southern)' -> 'TNXL Academy'
'Liberty HS' -> 'Liberty'
'John Carroll Catholic HS' -> 'Catholic HS'
'Brandon HS' -> 'Brandon'
'De La Salle HS' -> 'De La Salle'
'Franklin HS' -> 'Franklin'
'Pearland HS' -> 'Pearl'
'Columbia River' -> 'Columbia'
'Santa Margarita Catholic HS' -> 'Catholic HS'
'Buchanan HS' -> 'Buchanan'
'Sherwood HS' -> 'Sherwood'
'Centennial HS' -> 'Centennial'
'Columbia River HS' -> 'Columbia'
'Cardinal Newman HS' -> 'Cardinal Newman'
'Aquinas HS' -> 'Aquinas'
'Liberty-Wentzville' -> 'Liberty'
'East Ridge High School' -> 'East Ridge'
'Austin Prep' -> 'Austin'
'Wayzata HS' -> 'Wayzata'
'Montgomery Lake Creek' -> 'Lake Creek'
'Trussville' -> 'Hewitt-Trussville'
'TNXL Academy [Fla.]' -> 'TNXL Academy'


In [77]:
def dedup_coaches(coaches_df):
    # 1. John Smith at ABC Univ: Head Coach
    # 2. John Smith at ABC Univ: Recruiting Coordinator
    # => 1. John Smith at ABC Univ: Head Coach | Recruiting Coordinator
    coaches_clean = coaches_df.groupby(['Name', 'School'], as_index=False).agg({
        'Title': lambda x: ' | '.join(sorted(set(x)))
    })
    return coaches_clean

print(coaches.head())
coaches_clean = dedup_coaches(coaches)
print(coaches_clean)

                  School              Name                    Title
0  University Of Florida  Kevin O'Sullivan               Head Coach
1  University Of Florida   Chuck Jeroloman     Associate Head Coach
2  University Of Florida      Taylor Black          Assistant Coach
3  University Of Florida        David Kopp           Pitching Coach
4  University Of Florida     Michael Byrne  Student Assistant Coach
                 Name                   School  \
0       Adam  Pallone       Rutgers University   
1         Adam Nelson   University of Maryland   
2       Alec Crawford  University Of Minnesota   
3         Alex Corbin   University of Maryland   
4      Amanda Branson    University Of Alabama   
..                ...                      ...   
150  Tyler Schwalbach   University Of Oklahoma   
151     Vince Rinaldi   University Of Oklahoma   
152          Will Fox     Texas A&M University   
153       Zach Cronin    University Of Florida   
154  Zach Weatherford    University Of Ind

In [37]:
coaches = pd.read_csv('../data/raw/coaches.csv')
players = pd.read_csv('../data/raw/players.csv')
schools = pd.read_csv('../data/raw/schools.csv')

def normalize_school_name(name):
    if pd.isna(name) or str(name).strip().upper() == 'N/A':
        return None

    # Lowercase, remove punctuation, collapse whitespace
    name = str(name).lower()
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

def dedup_high_schools(players_df, schools_df, score_cutoff=80):
    canonical_hs = schools_df[schools_df['school type'] == 'high school']['name'].unique()
    canonical_hs_norm = [normalize_school_name(n) for n in canonical_hs if normalize_school_name(n)]

    hs_names = players_df['High School'].unique()

    hs_mapping = {}

    for orig in hs_names:
        norm = normalize_school_name(orig)
        if not norm:
            hs_mapping[orig] = orig
            continue
        matches = process.extract(
            norm,
            canonical_hs_norm,
            scorer=fuzz.token_sort_ratio,
            score_cutoff=score_cutoff,
        )
        if matches:
            idx = canonical_hs_norm.index(matches[0][0])
            hs_mapping[orig] = canonical_hs[idx]
            print(f"Fuzzy match: '{orig}' -> '{canonical_hs[idx]}' (score={matches[0][1]})")
        else:
            hs_mapping[orig] = orig
        
    players_df['High School'] = players_df['High School'].map(hs_mapping).fillna(players_df['High School'])
    schools_df['name'] = schools_df['name'].map(hs_mapping).fillna(schools_df['name'])
    
    schools_df.drop_duplicates(subset=['name'], inplace=True)

    # print(f"Original: {len(hs_names)}")
    # print(f"Canonical: {len(set(hs_mapping.values()))}")
    print(f"After deduplication: {len(hs_names) - len(set(hs_mapping.values()))} duplicates removed.")

    return players_df, schools_df

In [38]:
dedup_high_schools(players, schools)

Fuzzy match: 'Bartram Trail (Jacksonville)' -> 'Bartram Trail (Jacksonville)' (score=100.0)
Fuzzy match: 'Lakeland Christian' -> 'Lakeland Christian' (score=100.0)
Fuzzy match: 'North Oconee (Stetson)' -> 'North Oconee (Stetson)' (score=100.0)
Fuzzy match: 'Berkeley Prep' -> 'Berkeley Prep' (score=100.0)
Fuzzy match: 'Windermere (Miami)' -> 'Windermere (Miami)' (score=100.0)
Fuzzy match: 'Wiregrass Ranch (USF)' -> 'Wiregrass Ranch (USF)' (score=100.0)
Fuzzy match: 'Marjory Stoneman Douglas' -> 'Marjory Stoneman Douglas' (score=100.0)
Fuzzy match: 'Milton' -> 'Milton' (score=100.0)
Fuzzy match: 'Bloomingdale (Alabama)' -> 'Bloomingdale (Alabama)' (score=100.0)
Fuzzy match: 'P27 Academy' -> 'P27 Academy' (score=100.0)
Fuzzy match: 'Calvary Christian' -> 'Calvary Christian' (score=100.0)
Fuzzy match: 'Parkview (Texas Tech)' -> 'Parkview (Texas Tech)' (score=100.0)
Fuzzy match: 'A3 Academy' -> 'A3 Academy' (score=100.0)
Fuzzy match: 'Sickles' -> 'Sickles' (score=100.0)
Fuzzy match: 'Verona

(                    School             Name  Jersey Position Class Year  \
 0    University Of Florida    Justin Nadeau       1   INF/OF        Jr.   
 1    University Of Florida         Ty Evans       2       OF        Sr.   
 2    University Of Florida       Kyle Jones       3       OF        So.   
 3    University Of Florida     Cade Kurland       4      INF        Jr.   
 4    University Of Florida        Blake Cyr       5   OF/INF        Jr.   
 ..                     ...              ...     ...      ...        ...   
 486     Rutgers University    Justin Shadek      31      RHP      R-Fr.   
 487     Rutgers University     Jack Sweeney      34       OF        Fr.   
 488     Rutgers University      JT Thompson      14      INF        Jr.   
 489     Rutgers University  Mike Timberlake      20      RHP        Jr.   
 490     Rutgers University   Nick Tomasetto      23      INF      R-Fr.   
 
      Height Weight                   High School  
 0        72    195  Bartram Trail